In [1]:
!git clone https://github.com/Olyco/EasyTSF.git

Cloning into 'EasyTSF'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 266 (delta 131), reused 198 (delta 75), pack-reused 0 (from 0)
Receiving objects: 100% (266/266), 1.52 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [1]:
!pip install -r "EasyTSF/requirements.txt"

In [ ]:
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 MB 30.1 MB/s eta 0:00:00


In [ ]:
# !python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/RMoK/ETTh1_96for96.py

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type      | Params | Mode 
----------------------------------------------------
0 | model         | DenseRMoK | 130 K  | train
1 | loss_function | MSELoss   | 0      | train
----------------------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.520     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 132/132 [02:33<00:00,  1.16s/it, v_num=ed_1, train/loss_step=0.404]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation:   0% 0/44 [00:00<?, ?it/s]       
Validation DataLoader 0:   0% 0/44 [00:00<?, ?it/s]
Validation DataLoader 0:   2% 1/44 [00:00<00:13,  3.27it/s]
Validation DataLoader 0:   5% 2/44 [00:00<00:10,  3.90it/s]
Validation DataLoader 0:   7% 3/44 [00:00<00:09,  4.17it/s]
Validation DataLoader 0:   9

In [ ]:
!ls

EasyTSF  sample_data


In [10]:
!pwd

/content


In [12]:
config = dict(
    dataset_name='ETTh1',
    var_num=7,
    freq=60,
    data_split=[8640, 2880, 2880],

    hist_len=96,
    pred_len=96,

    batch_size=64,
    max_epochs=10,
    lr=0.0001,
    optimizer="AdamW",
    optimizer_betas=(0.95, 0.9),
    optimizer_weight_decay=1e-5,
    lr_scheduler='StepLR',
    lr_step_size=1,
    lr_gamma=0.5,
    gradient_clip_val=5,
    val_metric="val/loss",
    test_metric="test/mae",
    es_patience=10,

    norm_time_feature=False,
    time_feature_cls=["tod", "dow"],

    num_workers=2,

    model_name="DenseRMoK",

    revin_affine=True,

    data_root="EasyTSF/dataset",
)

In [20]:
from EasyTSF.easytsf.runner.data_runner import DataInterface

In [21]:
data_module = DataInterface(**config)

In [22]:
display(data_module)

In [24]:
import numpy as np
import os

In [27]:
data_path = os.path.join(config['data_root'], "{}.npz".format(config['dataset_name']))
data = np.load(data_path)
display(data)

NpzFile 'EasyTSF/dataset/ETTh1.npz' with keys: variable, timestamp, mean, std

In [32]:
display(data['mean'])

array([ 7.93774225,  2.02103866,  5.0797706 ,  0.74618588,  2.78176239,
        0.78845312, 17.1282617 ])

In [33]:
display(data['std'])

array([5.81274941, 2.09010465, 5.51879358, 1.92637927, 1.02352266,
       0.63023664, 9.17649102])

In [34]:
display(data['timestamp'])

array(['2016-07-01T00:00:00.000000000', '2016-07-01T01:00:00.000000000',
       '2016-07-01T02:00:00.000000000', ...,
       '2018-02-20T21:00:00.000000000', '2018-02-20T22:00:00.000000000',
       '2018-02-20T23:00:00.000000000'], dtype='datetime64[ns]')

In [37]:
len(data['variable'])

14400